# Erstellen und Zugriff auf SQLite-Datenbank mit Python

## Zielen:

* Eine Dateank zu erstellen
* Eine Telle zu erstellen
* Daten in die Table einzufügen
* Daten aus der Telle abzufragen
* Das Ergebnis in ein Pandas-Datframe abzurufen
* Die Verbindung zuratenbank z

SQLite ist eine Softwarebibliothek, die eine eigenständige, serverlose, konfigurationsfreie, transaktionale SQL-Datenbank-Engine implementiert. SQLite ist die am weitesten verbreitete SQL-Datenbank-Engine der Welt.

## Aufgabe 1: Erstellen einer Datenbank mit SQLite

In [1]:
import sqlite3

In [2]:
# Verbindung und Erstellung von sqlite Datenbank
# connection object
conn = sqlite3.connect('products.db')

Nun kriegen wir in unserem Verzeichnis eine Datei namens `products.db`

<img src='sqlite_db.png'>

Du kannst ein Cursor-Objekt erstellen, indem du die Methode `cursor()` des Verbindungsobjekts/-klassen aufrufst. Die Cursor-Klasse ist eine Instanz, die es ermöglicht, Methoden aufzurufen, die SQLite-Abfragen ausführen und Daten aus den Ergebnismengen der Abfragen abrufen.

In [3]:
# cursor object
cursor_obj = conn.cursor()

## Aufgabe 2: Erstellung einer Tabelle in der Datenbank


In [4]:
# Tabelle erstellen
table = """CREATE TABLE IF NOT EXISTS fahrrad (
                ID INTEGER PRIMARY KEY NOT NULL,
                Gewicht_kg INTEGER,
                Radgröße_Zoll INTEGER,
                Gänge INTEGER,
                Materialien VARCHAR(200),
                Fahrradtypen VARCHAR(30),
                Reifendruck_psi_bar VARCHAR(10),
                Price_EUR INTEGER
            );"""
cursor_obj.execute(table)

print("Tabelle ist bereit")

Tabelle ist bereit


### Tabellenliste der Datenbank products.db anzeigen

Dafür brauchen wir die  Systemtabelle `sqlite_master`-Tabelle abzufragen, die Infos über das Schema der Datenbank speichert

In [5]:
def show_all_tables():
    select_query = '''select name from sqlite_master  where type = 'table' '''
    cursor_obj.execute(select_query)
    tables = cursor_obj.fetchall()
    return tables

In [6]:
show_all_tables()

[('fahrrad',)]

In [7]:
# Infos über Tabelle anzeigen
def get_table_columns_info_df(table_name):
    def get_table_columns_info(table_name):
        query = "PRAGMA table_info({})".format(table_name)
        cursor_obj.execute(query)
        columns_info = cursor_obj.fetchall()

        columns_info_with_autoincrement = []
        for column_info in columns_info:
            cid, name, data_type, not_null, default_value, primary_key = column_info
            autoincrement = 'YES' if default_value and 'AUTOINCREMENT' in default_value else 'NO'
            columns_info_with_autoincrement.append((cid, name, data_type, not_null, default_value, primary_key, autoincrement))

        return columns_info_with_autoincrement
    # columns_info_with_autoincrement zu Dataframe umwandeln    
    import pandas as pd    
    columns_info = get_table_columns_info(table_name)
    column_names = ["Spalten_ID","Spaltenname", "Datentyp", "Zulässt NULL", "Standardwert", "Primärschlüssel", "Autoincrement"]
    df = pd.DataFrame(columns_info, columns=column_names)
    df.set_index('Spalten_ID', inplace=True)
    return df

# Infos über Tabelle "fahrrad" als DataFrame
get_table_columns_info_df('fahrrad')

,Spaltenname,Datentyp,Zulässt NULL,Standardwert,Primärschlüssel,Autoincrement
Spalten_ID,,,,,,
0,ID,INTEGER,1,None,1,NO
1,Gewicht_kg,INTEGER,0,None,0,NO
2,Radgröße_Zoll,INTEGER,0,None,0,NO
3,Gänge,INTEGER,0,None,0,NO
4,Materialien,VARCHAR(200),0,None,0,NO
5,Fahrradtypen,VARCHAR(30),0,None,0,NO
6,Reifendruck_psi_bar,VARCHAR(10),0,None,0,NO
7,Price_EUR,INTEGER,0,None,0,NO


## Aufgabe 3: Daten in die Tabelle einfügen


In [8]:
# Die Daten sollten als tuples eingegeben

### Manuel

In [9]:
# fügen wir die erste Reihe manuel ein
insert_query = '''insert into fahrrad values (21, 8, 29, "21-Gang", "Rahmen: Aluminium, Komponenten: Aluminiumlegierung", "Mountainbike", "30-50 psi (2-3,5 bar)", 1200)'''
cursor_obj.execute(insert_query)

#Änderungen bestätigen
conn.commit()

### Automatisch

In [10]:
# Die obere methode ist der Basic aber sie ist weniger praktisch dafür fügen wir die Daten auf einmal anhand einer Schleife und eine Funktion

Das sind Die Daten die wir einfügen werden:

In [11]:
data_to_insert = [
    (2, 12.5, 27.5, "7-Gang", "Rahmen: Stahl, Komponenten: Stahl", "Citybike", "50-70 psi (3,5-4,8 bar)", 800),
    (3, 9.8, 26, "Einzelgang", "Rahmen: Carbonfaser, Komponenten: Kohlefaser", "Rennrad", "80-120 psi (5,5-8,3 bar)", 1500),
    (4, 16, 26, "3-Gang", "Rahmen: Aluminium, Komponenten: Stahl", "Trekkingrad", "60-80 psi (4,1-5,5 bar)", 1000),
    (5, 12.7, 28, "18-Gang", "Rahmen: Aluminium, Komponenten: Titan", "Rennrad", "90-120 psi (6,2-8,3 bar)", 2000),
    (6, 10.3, 27.5, "10-Gang", "Rahmen: Aluminium, Komponenten: Kohlefaser", "Mountainbike", "35-55 psi (2,4-3,8 bar)", 1200),
    (7, 14.8, 26, "5-Gang", "Rahmen: Stahl, Komponenten: Aluminiumlegierung", "Citybike", "50-70 psi (3,5-4,8 bar)", 800),
    (8, 9.1, 29, "Einzelgang", "Rahmen: Carbonfaser, Komponenten: Carbonfaser", "Rennrad", "80-120 psi (5,5-8,3 bar)", 1500),
    (9, 13.5, 26, "7-Gang", "Rahmen: Aluminium, Komponenten: Stahl", "Trekkingrad", "60-80 psi (4,1-5,5 bar)", 1000),
    (10, 12.7, 28, "18-Gang", "Rahmen: Aluminium, Komponenten: Titan", "Rennrad", "90-120 psi (6,2-8,3 bar)", 2000),
    (11, 9.6, 29, "21-Gang", "Rahmen: Titan, Komponenten: Aluminiumlegierung", "E-Bike", "35-50 psi (2,4-3,5 bar)", 2500),
    (12, 15.2, 27.5, "7-Gang", "Rahmen: Stahl, Komponenten: Stahl", "Citybike", "45-65 psi (3,1-4,5 bar)", 700),
    (13, 10.9, 26, "3-Gang", "Rahmen: Aluminium, Komponenten: Aluminiumlegierung", "Trekkingrad", "60-80 psi (4,1-5,5 bar)", 900),
    (14, 11.8, 28, "Einzelgang", "Rahmen: Carbonfaser, Komponenten: Carbonfaser", "Rennrad", "90-120 psi (6,2-8,3 bar)", 1800),
    (15, 13.4, 29, "21-Gang", "Rahmen: Aluminium, Komponenten: Titan", "E-Bike", "35-50 psi (2,4-3,5 bar)", 2800),
    (16, 12.1, 27.5, "10-Gang", "Rahmen: Titan, Komponenten: Kohlefaser", "Mountainbike", "35-55 psi (2,4-3,8 bar)", 1300),
    (17, 14.6, 26, "7-Gang", "Rahmen: Stahl, Komponenten: Stahl", "Citybike", "50-70 psi (3,5-4,8 bar)", 750),
    (18, 10.2, 29, "Einzelgang", "Rahmen: Aluminium, Komponenten: Carbonfaser", "Rennrad", "80-120 psi (5,5-8,3 bar)", 1600),
    (19, 13.8, 26, "5-Gang", "Rahmen: Stahl, Komponenten: Aluminiumlegierung", "Trekkingrad", "60-80 psi (4,1-5,5 bar)", 1100),
    (20, 11.5, 28, "18-Gang", "Rahmen: Aluminium, Komponenten: Titan", "Rennrad", "90-120 psi (6,2-8,3 bar)", 2200)
]

In [12]:
# Funkrion um Daten einzufügen hier habe ich try except Ausnahmen benutzt das ist nicht notwendig aber hilfreich um Fehler leicht zu erkennen 

In [13]:
def insert_data(id, weight, wheel_size, gears, materials, bike_types, tire_pressure_psi_bar, price):
    try:
        cursor_obj.execute('''INSERT INTO fahrrad (ID, Gewicht_kg, Radgröße_Zoll, Gänge, Materialien, Fahrradtypen, Reifendruck_psi_bar, Price_EUR)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', 
                           (id, weight, wheel_size, gears, materials, bike_types, tire_pressure_psi_bar, price))
        # Änderungen bestätigen
        conn.commit()
        print("Daten erfolgreich eingefügt:", id, weight, wheel_size, gears, materials , bike_types, tire_pressure_psi_bar, price)
    except sqlite3.IntegrityError:
        print(f"Fehler:ID={id} Eintrag mit dieser ID existiert bereits.")
    except Exception as e:
        print("Allgemeiner Fehler:", e)

In [14]:
for row in data_to_insert:
    insert_data(*row)

Daten erfolgreich eingefügt: 2 12.5 27.5 7-Gang Rahmen: Stahl, Komponenten: Stahl Citybike 50-70 psi (3,5-4,8 bar) 800
Daten erfolgreich eingefügt: 3 9.8 26 Einzelgang Rahmen: Carbonfaser, Komponenten: Kohlefaser Rennrad 80-120 psi (5,5-8,3 bar) 1500
Daten erfolgreich eingefügt: 4 16 26 3-Gang Rahmen: Aluminium, Komponenten: Stahl Trekkingrad 60-80 psi (4,1-5,5 bar) 1000
Daten erfolgreich eingefügt: 5 12.7 28 18-Gang Rahmen: Aluminium, Komponenten: Titan Rennrad 90-120 psi (6,2-8,3 bar) 2000
Daten erfolgreich eingefügt: 6 10.3 27.5 10-Gang Rahmen: Aluminium, Komponenten: Kohlefaser Mountainbike 35-55 psi (2,4-3,8 bar) 1200
Daten erfolgreich eingefügt: 7 14.8 26 5-Gang Rahmen: Stahl, Komponenten: Aluminiumlegierung Citybike 50-70 psi (3,5-4,8 bar) 800
Daten erfolgreich eingefügt: 8 9.1 29 Einzelgang Rahmen: Carbonfaser, Komponenten: Carbonfaser Rennrad 80-120 psi (5,5-8,3 bar) 1500
Daten erfolgreich eingefügt: 9 13.5 26 7-Gang Rahmen: Aluminium, Komponenten: Stahl Trekkingrad 60-80 psi 

In [15]:
# Der id soll eindeutig sein sowie wir es bei der Erstellung der Tabelle definiert 

## Aufgabe 4: Fetch (Abruf) der Daten aus der Tabelle

In [16]:
# Alle Daten der Tabelle `fahrrad` anzeigen
select_query = '''SELECT * FROM fahrrad'''
cursor_obj.execute(select_query)
# Jetzt das cursor_obj  enthält Daten der Tabelle fahrrad sei es Spaltennamen oder Values aber in eine unordentliche Struktur.

### Values in Form einer Liste von tuples abrufen

In [17]:
# Dafür brauchen wir fetch methoden wie z.b. fetchall() um diese Daten in Form einer Liste von tuples zuzuordnen
values_all = cursor_obj.fetchall()

In [18]:
type(values_all)

list

In [19]:
# Zwei Elemente der Liste output_all anzeigen
values_all[0:2]

[(2,
  12.5,
  27.5,
  '7-Gang',
  'Rahmen: Stahl, Komponenten: Stahl',
  'Citybike',
  '50-70 psi (3,5-4,8 bar)',
  800),
 (3,
  9.8,
  26,
  'Einzelgang',
  'Rahmen: Carbonfaser, Komponenten: Kohlefaser',
  'Rennrad',
  '80-120 psi (5,5-8,3 bar)',
  1500)]

In [20]:
# Nun können wir ganz ruhig unsere Daten anzeigen anhand einer Schleife
for tuple in values_all:
    print(tuple)

(2, 12.5, 27.5, '7-Gang', 'Rahmen: Stahl, Komponenten: Stahl', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(3, 9.8, 26, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Kohlefaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(4, 16, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)
(5, 12.7, 28, '18-Gang', 'Rahmen: Aluminium, Komponenten: Titan', 'Rennrad', '90-120 psi (6,2-8,3 bar)', 2000)
(6, 10.3, 27.5, '10-Gang', 'Rahmen: Aluminium, Komponenten: Kohlefaser', 'Mountainbike', '35-55 psi (2,4-3,8 bar)', 1200)
(7, 14.8, 26, '5-Gang', 'Rahmen: Stahl, Komponenten: Aluminiumlegierung', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(8, 9.1, 29, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Carbonfaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(9, 13.5, 26, '7-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)
(10, 12.7, 28, '18-Gang', 'Rahmen: Aluminium, Komponenten: Titan', 'Rennrad',

In [21]:
# Values mit Spaltennamen anzeigen 
# Spaltennamen abrufen
column_names = [description[0] for description in cursor_obj.description]
print(column_names)
for tuple in values_all:
    print(tuple)

['ID', 'Gewicht_kg', 'Radgröße_Zoll', 'Gänge', 'Materialien', 'Fahrradtypen', 'Reifendruck_psi_bar', 'Price_EUR']
(2, 12.5, 27.5, '7-Gang', 'Rahmen: Stahl, Komponenten: Stahl', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(3, 9.8, 26, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Kohlefaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(4, 16, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)
(5, 12.7, 28, '18-Gang', 'Rahmen: Aluminium, Komponenten: Titan', 'Rennrad', '90-120 psi (6,2-8,3 bar)', 2000)
(6, 10.3, 27.5, '10-Gang', 'Rahmen: Aluminium, Komponenten: Kohlefaser', 'Mountainbike', '35-55 psi (2,4-3,8 bar)', 1200)
(7, 14.8, 26, '5-Gang', 'Rahmen: Stahl, Komponenten: Aluminiumlegierung', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(8, 9.1, 29, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Carbonfaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(9, 13.5, 26, '7-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingra

## Aufgabe 5: Daten als ein Dataframe abrufen

In [22]:
# Diese Tabelle können wir auch als Dataframe anhand der methode read_sql abrufen
import pandas as pd
all_data = '''select * from fahrrad'''
cursor_obj.execute(all_data)
df= pd.read_sql(all_data, con=conn)
df

,ID,Gewicht_kg,Radgröße_Zoll,Gänge,Materialien,Fahrradtypen,Reifendruck_psi_bar,Price_EUR
0,2,12.5,27.5,7-Gang,"Rahmen: Stahl, Komponenten: Stahl",Citybike,"50-70 psi (3,5-4,8 bar)",800
1,3,9.8,26.0,Einzelgang,"Rahmen: Carbonfaser, Komponenten: Kohlefaser",Rennrad,"80-120 psi (5,5-8,3 bar)",1500
2,4,16.0,26.0,3-Gang,"Rahmen: Aluminium, Komponenten: Stahl",Trekkingrad,"60-80 psi (4,1-5,5 bar)",1000
3,5,12.7,28.0,18-Gang,"Rahmen: Aluminium, Komponenten: Titan",Rennrad,"90-120 psi (6,2-8,3 bar)",2000
4,6,10.3,27.5,10-Gang,"Rahmen: Aluminium, Komponenten: Kohlefaser",Mountainbike,"35-55 psi (2,4-3,8 bar)",1200
5,7,14.8,26.0,5-Gang,"Rahmen: Stahl, Komponenten: Aluminiumlegierung",Citybike,"50-70 psi (3,5-4,8 bar)",800
6,8,9.1,29.0,Einzelgang,"Rahmen: Carbonfaser, Komponenten: Carbonfaser",Rennrad,"80-120 psi (5,5-8,3 bar)",1500
7,9,13.5,26.0,7-Gang,"Rahmen: Aluminium, Komponenten: Stahl",Trekkingrad,"60-80 psi (4,1-5,5 bar)",1000
8,10,12.7,28.0,18-Gang,"Rahmen: Aluminium, Komponenten: Titan",Rennrad,"90-120 psi (6,2-8,3 bar)",2000
9,11,9.6,29.0,21-Gang,"Rahmen: Titan, Komponenten: Aluminiumlegierung",E-Bike,"35-50 psi (2,4-3,5 bar)",2500


## Tabelle bearbeiten

Dafür haben eine Lise von Auswahl  unter anderem  sql-Anweisungen und pandas.
Persönlich empfehle ich pandas am ehesten da ihr Syntax ganz easy ist und man can alles schaffen was sql kann in weniger Zeilen.

In diesem Skript werde ich trotzdem sql-Anweisungen verwenden um zu üben wie man sql-anweisungen mit python verwenden

### 1.  Fetch Data

 Es gibt verschiede methode von fetch data unter anderem
- fetchall() ruft alle Daten ab
- fetchmany(n) ruft n ersten Elemente ab
- usw...

In [23]:
# Fetch nur Fahrradtypen aus der Tabelle
fetch_query = '''SELECT Fahrradtypen FROM fahrrad'''
cursor_obj.execute(fetch_query)

bicycle_type = cursor_obj.fetchall()
for tuple in bicycle_type:
  print(tuple)

('Citybike',)
('Rennrad',)
('Trekkingrad',)
('Rennrad',)
('Mountainbike',)
('Citybike',)
('Rennrad',)
('Trekkingrad',)
('Rennrad',)
('E-Bike',)
('Citybike',)
('Trekkingrad',)
('Rennrad',)
('E-Bike',)
('Mountainbike',)
('Citybike',)
('Rennrad',)
('Trekkingrad',)
('Rennrad',)
('Mountainbike',)


In [24]:
# Nur erste fünf Element aus der Spalte Fahrradtypen zeigen
fetch_query = '''SELECT Fahrradtypen FROM fahrrad limit 5'''
cursor_obj.execute(fetch_query)

bicycle_type_top_5 = cursor_obj.fetchall()
for tuple in bicycle_type_top_5:
    print(tuple)

('Citybike',)
('Rennrad',)
('Trekkingrad',)
('Rennrad',)
('Mountainbike',)


In [25]:
 # wir können den aktuellen Zustand der Tabelle nur in einer Zeile anzeigen anhand eine Funktion

def show_table():
    fetch_query = '''SELECT * FROM fahrrad'''
    cursor_obj.execute(fetch_query)

    # Spaltennamen abrufen
    column_names = [description[0] for description in cursor_obj.description]
    print(column_names)
    
    # Values abrufen
    all_data = cursor_obj.fetchall()
    for tuple in all_data:
        print(tuple)

In [26]:
# aktuelle Zustand der Tabelle anzeigen
show_table()

['ID', 'Gewicht_kg', 'Radgröße_Zoll', 'Gänge', 'Materialien', 'Fahrradtypen', 'Reifendruck_psi_bar', 'Price_EUR']
(2, 12.5, 27.5, '7-Gang', 'Rahmen: Stahl, Komponenten: Stahl', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(3, 9.8, 26, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Kohlefaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(4, 16, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)
(5, 12.7, 28, '18-Gang', 'Rahmen: Aluminium, Komponenten: Titan', 'Rennrad', '90-120 psi (6,2-8,3 bar)', 2000)
(6, 10.3, 27.5, '10-Gang', 'Rahmen: Aluminium, Komponenten: Kohlefaser', 'Mountainbike', '35-55 psi (2,4-3,8 bar)', 1200)
(7, 14.8, 26, '5-Gang', 'Rahmen: Stahl, Komponenten: Aluminiumlegierung', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(8, 9.1, 29, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Carbonfaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(9, 13.5, 26, '7-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingra

In [27]:
# Wir können auch der Anzahl der ersten angezeigten Zeilen einschränken durch die methode fetchmany()
def select_first_n_rows(n):
    fetch_query = '''SELECT * FROM fahrrad'''
    cursor_obj.execute(fetch_query)

    all_data = cursor_obj.fetchmany(n)
    for tuple in all_data:
        print(tuple)

In [28]:
select_first_n_rows(3)

(2, 12.5, 27.5, '7-Gang', 'Rahmen: Stahl, Komponenten: Stahl', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(3, 9.8, 26, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Kohlefaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(4, 16, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)


In [29]:
select_first_n_rows(4)

(2, 12.5, 27.5, '7-Gang', 'Rahmen: Stahl, Komponenten: Stahl', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(3, 9.8, 26, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Kohlefaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(4, 16, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)
(5, 12.7, 28, '18-Gang', 'Rahmen: Aluminium, Komponenten: Titan', 'Rennrad', '90-120 psi (6,2-8,3 bar)', 2000)


In [30]:
def select_random_rows(n):
    import random
    fetch_query = '''SELECT * FROM fahrrad ORDER BY RANDOM() LIMIT ?'''
    cursor_obj.execute(fetch_query, (n,))
    
    all_data = cursor_obj.fetchall()
    for row in all_data:
        print(row)

In [31]:
select_random_rows(4)

(13, 10.9, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Aluminiumlegierung', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 900)
(18, 10.2, 29, 'Einzelgang', 'Rahmen: Aluminium, Komponenten: Carbonfaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1600)
(6, 10.3, 27.5, '10-Gang', 'Rahmen: Aluminium, Komponenten: Kohlefaser', 'Mountainbike', '35-55 psi (2,4-3,8 bar)', 1200)
(4, 16, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)


### 2. Update data

In [32]:
# Den Preis des sechsten Fahrrad  von 1200 zu 1250 ändern
update_query ='''update fahrrad set Price_EUR=1250 where ID=6'''
cursor_obj.execute(update_query)

In [33]:
# aktuelle Zustand der Tabelle anzeigen
show_table()

['ID', 'Gewicht_kg', 'Radgröße_Zoll', 'Gänge', 'Materialien', 'Fahrradtypen', 'Reifendruck_psi_bar', 'Price_EUR']
(2, 12.5, 27.5, '7-Gang', 'Rahmen: Stahl, Komponenten: Stahl', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(3, 9.8, 26, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Kohlefaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(4, 16, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)
(5, 12.7, 28, '18-Gang', 'Rahmen: Aluminium, Komponenten: Titan', 'Rennrad', '90-120 psi (6,2-8,3 bar)', 2000)
(6, 10.3, 27.5, '10-Gang', 'Rahmen: Aluminium, Komponenten: Kohlefaser', 'Mountainbike', '35-55 psi (2,4-3,8 bar)', 1250)
(7, 14.8, 26, '5-Gang', 'Rahmen: Stahl, Komponenten: Aluminiumlegierung', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(8, 9.1, 29, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Carbonfaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(9, 13.5, 26, '7-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingra

In [34]:
# Der Preis des 6. ten Fahrrad ist schon um 1250 geändert

### 3 Delete Data

In [35]:
# Zeilen 5 bis 15 entfernen
delete_query = '''delete from fahrrad where ID >=5 and ID <=15 '''
cursor_obj.execute(delete_query)

In [36]:
# Aktualisierte Tabelle zeigen
show_table()

['ID', 'Gewicht_kg', 'Radgröße_Zoll', 'Gänge', 'Materialien', 'Fahrradtypen', 'Reifendruck_psi_bar', 'Price_EUR']
(2, 12.5, 27.5, '7-Gang', 'Rahmen: Stahl, Komponenten: Stahl', 'Citybike', '50-70 psi (3,5-4,8 bar)', 800)
(3, 9.8, 26, 'Einzelgang', 'Rahmen: Carbonfaser, Komponenten: Kohlefaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1500)
(4, 16, 26, '3-Gang', 'Rahmen: Aluminium, Komponenten: Stahl', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1000)
(16, 12.1, 27.5, '10-Gang', 'Rahmen: Titan, Komponenten: Kohlefaser', 'Mountainbike', '35-55 psi (2,4-3,8 bar)', 1300)
(17, 14.6, 26, '7-Gang', 'Rahmen: Stahl, Komponenten: Stahl', 'Citybike', '50-70 psi (3,5-4,8 bar)', 750)
(18, 10.2, 29, 'Einzelgang', 'Rahmen: Aluminium, Komponenten: Carbonfaser', 'Rennrad', '80-120 psi (5,5-8,3 bar)', 1600)
(19, 13.8, 26, '5-Gang', 'Rahmen: Stahl, Komponenten: Aluminiumlegierung', 'Trekkingrad', '60-80 psi (4,1-5,5 bar)', 1100)
(20, 11.5, 28, '18-Gang', 'Rahmen: Aluminium, Komponenten: Titan', 'Rennrad', '

In [37]:
# Alle Zeile entleeren
truncate_query = '''DELETE FROM fahrrad'''
cursor_obj.execute(truncate_query)

# Änderungen bestätigen
conn.commit()

In [38]:
show_table

<function __main__.show_table()>

### 4. Delete Table 

In [39]:
drop_table_query = '''DROP TABLE IF EXISTS fahrrad'''
cursor_obj.execute(drop_table_query)
conn.commit()

In [40]:
show_all_tables()

[]

## Aufgabe 5: Die Verbindung schließen

Wir geben alle Ressourcen frei, indem wir die Verbindung schließen. Denke daran, dass es immer wichtig ist, Verbindungen zu schließen, um zu vermeiden, dass ungenutzte Verbindungen Ressourcen verbrauchen.

In [41]:
# Close the connection
conn.close()